In [ ]:
import numpy as np
import os
import glob
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import bat_functions as bf

In [ ]:
folder = ".../kasanka-bats/gopros/17Nov"

for f in glob.glob(os.path.join(folder, '*')):
    files = glob.glob(os.path.join(f, '*.MP4'))
    
    filename = os.path.basename(files[0])
    if filename[:2] == 'GP':
        print(os.path.basename(f))
        print(os.path.basename(files[0]))

In [ ]:
fps = 25

In [ ]:
plots_save_folder = '.../bats-data/plots/track-behavior'
os.makedirs(plots_save_folder, exist_ok=True)

In [ ]:
day_folders = sorted(
    glob.glob(
        '.../kasanka-bats/processed/deep-learning/*Nov'
    )
)

min_thresh = 100

all_tracks = {}
for day_folder in day_folders[:]:
    print(day_folder)
    date = os.path.basename(day_folder)
    track_files = sorted(
        glob.glob(
            os.path.join(day_folder, 
                         f'*/long_tracks_min_{min_thresh}_wingbeat.npy'
                        )
        )
    )
    all_tracks[date] = {}
    for t_ind, track_file in enumerate(track_files):
        camera = track_file.split('/')[-2]
        tracks = np.load(track_file, allow_pickle=True)
        all_tracks[date][camera] = tracks

_ = all_tracks['17Nov'].pop('MusoleParking', None)
_ = all_tracks['18Nov'].pop('MusolePath', None)
_ = all_tracks['20Nov'].pop('MusolePath', None)

In [ ]:
colormap = plt.cm.tab10
camera_colors = bf.get_camera_color_dict(colormap)

In [ ]:
all_tracks.keys()

In [ ]:
save = False

### Height

In [ ]:
shift = 48
HCONST = 1454.9 # pixels
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

for date, day_tracks in all_tracks.items():
    for camera, tracks in day_tracks.items():
        for track in tracks:
            height = bf.calculate_height(track['mean_wing'], HCONST, WINGSPAN)
            track['height'] = height

In [ ]:
constant = WINGSPAN * HCONST
print(constant)
wing_pixels = np.arange(10, 300)
plt.plot(constant / wing_pixels)

plt.xlabel('Wingspan (pixels)')
plt.ylabel('Estimated height (meters)')

title = "Wingspan in pixels versus estimated height meters"
if save:
    bf.save_fig(plots_save_folder, title)

In [ ]:
camera_names = {'16Nov':['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '17Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking2', 'FibwePublic',
                         'MusoleTower', 'MusolePath2', 
                         'Sunset', 'Puku'],
                '18Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
                '19Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '20Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
               }

In [ ]:
lower_percentile=.05
upper_percentile=.95

total = 0

total_tracks = 0

for date, day_tracks in all_tracks.items():
#     camera_names = []
    heights = []
    for t_ind, (camera, tracks) in enumerate(day_tracks.items()):
#         camera_names.append(camera)
        camera_heights = []
        for track in tracks:
            total_tracks += 1
            camera_heights.append(track['height'])
        camera_heights = bf.get_middle_percentiles(camera_heights,
                                               lower_percentile,
                                               upper_percentile)
        heights.extend(camera_heights)
    total += len(heights)
    print(f"{date} number of tracks {len(heights)}, .9 {len(heights) * .9}")
print(f"total {total} out of {total_tracks} .9: {.9 * total_tracks}")

In [ ]:
lower_percentile = .05
upper_percentile = .95

plt.rcParams.update({'font.size': 15})

save = False

max_height = 0
max_y = 160

for date, day_tracks in all_tracks.items():
#     camera_names = []
    heights = []
    xs = []
    colors = []
    for t_ind, camera in enumerate(camera_names[date]):
        tracks = day_tracks[camera]
#     for t_ind, (camera, tracks) in enumerate(day_tracks.items()):
#         camera_names.append(camera)
        camera_heights = []
        for track in tracks:
            camera_heights.append(track['height'])
        camera_heights = bf.get_middle_percentiles(camera_heights,
                                               lower_percentile,
                                               upper_percentile)
        xs.extend([t_ind for _ in camera_heights])
        heights.extend(camera_heights)
        colors.append(camera_colors[camera])
    fig, (ax1) = plt.subplots(1, 1)
    if max(heights) > max_height:
        max_height = max(heights)
    palette = sns.color_palette(colors)
    sns.violinplot(x=xs, y=heights, ax=ax1, palette=palette)
    ax1.set_title(f"{date}")
    ax1.set_xticklabels(camera_names[date], rotation=90)
    ax1.set_ylabel("Height (meters)")
    ax1.set_ylim(0, max_y)
    
    title = (f"{date} height distributions "
             + f"lower percentile {lower_percentile} "
             + f"upper percentile {upper_percentile} "
             + f"max yaxis {max_y}")

    if save:
        bf.save_fig(plots_save_folder, title)

print(max_height)

In [ ]:
np.max(heights)

In [ ]:
colors

In [ ]:
lower_percentile = .05
upper_percentile = .95

save = True
plt.rcParams.update({'font.size': 15})

max_freq = 0

for date, day_tracks in all_tracks.items():
    freqs = []
    xs = []
    colors = []
    for t_ind, camera in enumerate(camera_names[date]):
        tracks = day_tracks[camera]
        camera_freqs = []
        colors.append(camera_colors[camera])
        for track in tracks:
            camera_freqs.append(track['peak_freq'])
        camera_freqs = bf.get_middle_percentiles(camera_freqs,
                                               lower_percentile,
                                               upper_percentile)
        
        if max_freq < np.max(camera_freqs):
            max_freq = np.max(camera_freqs)
        xs.extend([t_ind for _ in camera_freqs])
        freqs.extend(camera_freqs)
    fig, (ax1) = plt.subplots(1, 1)

    palette = sns.color_palette(colors)
    sns.violinplot(x=xs, y=freqs, ax=ax1, palette=palette)
    ax1.set_title(f"{date}")
    ax1.set_xticklabels(camera_names[date], rotation=90)
    ax1.set_ylabel("Wingbeats per second")
    ax1.set_ylim(0, 5.0)
    
    title = (f"{date} wingbeat distributions "
             + f"lower percentile {lower_percentile} "
             + f"upper percentile {upper_percentile}"
             + f"big font")
    if save:
        bf.save_fig(plots_save_folder, title)
print(max_freq)

In [ ]:
save = False

lower_percentile = .05
upper_percentile = .95

for date_ind, (date, day_tracks) in enumerate(all_tracks.items()):
    colors = []
    straightness_measures = []
    all_cameras = []
    xs = []
    camera_names = []
    for cam_ind, (camera, tracks) in enumerate(day_tracks.items()):
        camera_names.append(camera)
        camera_straightness = []

        for track in tracks:
            camera_straightness.append(
                bf.calculate_straightness(track['track']))
        camera_straightness = bf.get_middle_percentiles(camera_straightness,
                                                     lower_percentile,
                                                     upper_percentile)
        straightness_measures.extend(camera_straightness)
        xs.extend([cam_ind for _ in camera_straightness])
        colors.append(camera_colors[camera])
        

    fig, ax = plt.subplots(1, 1)
    palette = sns.color_palette(colors)
#     sns.violinplot(x=xs, y=heights, ax=ax1, )
    sns.violinplot(x=xs, y=straightness_measures, 
                   ax=ax, cut=0, palette=palette)
    ax.set_title(f"{date}")
    ax.set_xticklabels(camera_names, rotation=90)
    ax.set_ylabel("Track Straightness")
    
    title = (f"{date} track straightness "
             + f"lower percentile {lower_percentile} "
             + f"upper percentile {upper_percentile}")
    if save:
        bf.save_fig(plots_save_folder, title)

In [ ]:
len(tracks)

In [ ]:
for track in tracks[4::500]:
    plt.figure(figsize=(5,5))
    camera_straightness = bf.calculate_straightness(track['track'])
    plt.title(camera_straightness)
    plt.scatter(track['track'][:,0], track['track'][:,1], s=.1)
    plt.gca().axis('equal')

    

In [ ]:
shift = 0 # loss on each side from not padding during detection (48)
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

center_utm = {'middle': np.array([200450, 8606950]),
              'right': np.array([200800, 8606900])}

camera_locations = {'FibweParking2': [-12.5903393, 30.2525047],	
                    'FibweParking': [-12.5903393, 30.2525047],
                    'Chyniangale': [-12.5851284, 30.245529],	
                    'BBC': [-12.5863538, 30.2484985],
                    'Sunset': [-12.585784, 30.240003],
                    'NotChyniangale': [-12.5849206,	30.2436135],
                    'MusoleParking': [-12.58787, 30.2401],	
                    'MusolePath2': [-12.589544,	30.242488],	
                    'MusolePath': [-12.589544,	30.242488],
                    'Puku': [-12.584838, 30.24137],	
                    'FibwePublic': [-12.592537, 30.2515924],	
                    'MusoleTower': [-12.589434, 30.244736],
                    }

all_camera_utms = bf.latlong_dict_to_utm(camera_locations)

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
observations_root = os.path.join(root_folder, "observations")
all_observations = {}
day_folders = sorted(glob.glob(os.path.join(observations_root, '*')))
for day_folder in day_folders:
    obs_files = sorted(glob.glob(os.path.join(day_folder, '*.npy')))
    date = os.path.basename(day_folder)
    all_observations[date] = {}
    for obs_file in obs_files:
        camera = os.path.splitext(obs_file)[0].split('-')[-1]
        obs = np.load(obs_file, allow_pickle=True)
        # .item() to get dict from inside the array that was wrapped around
        # it when using np.save()
        all_observations[date][camera] = obs.item()
        
# Remove observations to exclude (because camera ran out of batteries etc.)
exclude=True
# Manually exclude cameras that had issues
all_observations['17Nov']['MusoleParking']['exclude'] = True
all_observations['18Nov']['MusolePath']['exclude'] = True
all_observations['20Nov']['MusolePath']['exclude'] = True
if exclude:
    good_obs = {}
    for date, day_obs in all_observations.items():
        good_obs[date] = {}
        for camera, obs in day_obs.items():
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    continue
            good_obs[date][camera] = obs
    all_observations = good_obs

In [ ]:
axis_labels_day_ind = 0
save=False




center = 'middle'

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats_per_degree = 0
max_camera = None
max_date = None

min_bats_per_degree = 100000
min_camera = None
min_date = None

max_height = 0


for day_ind, ((date, day), fig, axs) in enumerate(zip(all_observations.items(), all_figs, all_axs)):


    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    fractions = []
    cameras = []
    totals = []
    contribution = []
    angles = []
    colors = []
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    
    print(date, len(camera_utms))
    
    day_densities = []

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    for camera, obs in day.items():
        if exclude:
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    print('excluding observation...')
                    continue
        fractions.append(obs['fraction_total'])
        cameras.append(camera)
        density = obs['total'] / 360
        totals.append(density)
        angles.append(camera_angles[camera])
        colors.append(camera_colors[camera])
        day_density.append(density)

        if density > max_bats_per_degree:
            max_bats_per_degree = density
            max_camera = camera
            max_date = date

        if density < min_bats_per_degree:
            min_bats_per_degree = density
            min_camera = camera
            min_date = date

    print()
    total_ind = 0
    axs.set_thetalim(-np.pi, np.pi)
    axs.scatter(angles, totals, color=colors)
#     if axis_labels_day_ind == day_ind:
    axs.set_xticks(angles)
    _ = axs.set_xticklabels(cameras, fontweight='bold')
    axs.set_title(f"{date}")
    
    axs.set_rlim(0, 12000)

    axs.text(np.radians(0), 
             3*axs.get_rmax()/4.,
             'Bats per degree',
            rotation=0, 
            ha='center', va='center')
    _ = axs.set_rticks([4000, 8000])
    
    

    title = f"{date} bat density "

    if save:
        bf.save_fig(plots_save_folder, title, fig=fig)
    
    
print(f"min camera {min_camera}, {min_date}: {min_bats_per_degree}")
print(f"max camera {max_camera}, {max_date}: {max_bats_per_degree}")

In [ ]:
camera_angles

In [ ]:
axis_labels_day_ind = 0
save=True




center = 'middle'

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats_per_degree = 0
max_bats_per_camera = 0
max_height = 0


for day_ind, ((date, day), fig, axs) in enumerate(zip(all_observations.items(), all_figs, all_axs)):


    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    fractions = []
    cameras = []
    distances = []
    angles = []
    colors = []
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    
    print(date, len(camera_utms))

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    camera_distances = bf.get_camera_distances(camera_utms, center_utm[center])
    for camera, obs in day.items():
        colors.append(camera_colors[camera])
        cameras.append(camera)
        distances.append(camera_distances[camera])
        angles.append(camera_angles[camera])
    total_ind = 0
    axs.set_thetalim(-np.pi, np.pi)
    axs.scatter(angles, distances, color=colors)
#     if axis_labels_day_ind == day_ind:
    axs.set_xticks(angles)
    _ = axs.set_xticklabels(cameras, fontweight='bold')
    axs.set_title(f"{date}")
    
#     axs.set_rlim(0, 12000)

    axs.text(np.radians(0), 
             3*axs.get_rmax()/4.,
             'Meters',
            rotation=0, 
            ha='center', va='center',
            color='gray')
    _ = axs.set_rticks([400, 800, 1200])
    

    axs.tick_params(axis='x', colors='gray')    
    axs.tick_params(axis='y', colors='gray')  #setting up Y-axis tick color to black

    axs.spines['polar'].set_color('gray')


    

    title = f"{date} camera distance gray"

    if save:
        bf.save_fig(plots_save_folder, title, fig=fig)

In [ ]:
axis_labels_day_ind = 0
save=True




center = 'middle'

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats = 0
max_camera = None
max_date = None

min_bats = 100000
min_camera = None
min_date = None

max_height = 0


for day_ind, ((date, day), fig, axs) in enumerate(zip(all_observations.items(), all_figs, all_axs)):


    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    fractions = []
    cameras = []
    totals = []
    contribution = []
    angles = []
    colors = []
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    
    print(date, len(camera_utms))
    
    day_density = []

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    for camera, obs in day.items():
        if exclude:
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    print('excluding observation...')
                    continue
        fractions.append(obs['fraction_total'])
        cameras.append(camera)
        camera_total = obs['total'] / 360 #* obs['fraction_total']
        totals.append(camera_total)
        angles.append(camera_angles[camera])
        colors.append(camera_colors[camera])
        day_density.append(density)

        if camera_total > max_bats:
            max_bats = camera_total
            max_camera = camera
            max_date = date

        if camera_total < min_bats:
            min_bats = camera_total
            min_camera = camera
            min_date = date

    total_ind = 0
    axs.set_thetalim(-np.pi, np.pi)
#     axs.scatter(angles, totals, color=colors)
    axs.bar(angles, totals)
#     if axis_labels_day_ind == day_ind:
    axs.set_xticks(angles)
    angles = np.array(angles)
    angles_compass = (90-angles*180/np.pi).astype(int)
    angles_compass = np.where(angles_compass < 0, angles_compass+360, angles_compass)
    angles_compass = [str(a) + u"\u00b0" for a in angles_compass]


    _ = axs.set_xticklabels(angles_compass, fontweight='bold')
    axs.set_title(f"{date}")
    
    axs.set_rlim(0, max_bats+500)

    axs.text(np.radians(50), 
             3*axs.get_rmax()/4.,
             'Bats per degree',
            rotation=50, 
            ha='center', va='center')
    _ = axs.set_rticks([3000, 6000, 9000])
    
    

    title = f"{date} bats per degree per camera radial histogram degree labels"

    if save:
        bf.save_fig(plots_save_folder, title, fig=fig)
    
    
print(f"min camera {min_camera}, {min_date}: {min_bats_per_degree}")
print(f"max camera {max_camera}, {max_date}: {max_bats_per_degree}")

In [ ]:
axis_labels_day_ind = 0
save=True




center = 'middle'

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats = 0
max_camera = None
max_date = None

min_bats = 100000
min_camera = None
min_date = None

max_height = 0


for day_ind, ((date, day), fig, axs) in enumerate(zip(all_observations.items(), all_figs, all_axs)):


    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    
    densities = np.ones(360)
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    camera_borders = bf.get_camera_borders(camera_utms, center_utm[center])
    
    print(date, len(camera_utms))
    

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    for camera, obs in day.items():
        if exclude:
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    print('excluding observation...')
                    continue
        fractions.append(obs['fraction_total'])
        cameras.append(camera)
        camera_total = obs['total'] / 360 #* obs['fraction_total']
        lower = camera_angles[camera] - camera_borders[camera]['clock_angle']
        upper = camera_angles[camera] - camera_borders[camera]['cclock_angle']
        
        lower = lower * 180 / np.pi
        upper = upper * 180 / np.pi
        lower = (lower).astype(int)
        lower = np.where(lower < 0, lower+360, lower)
        upper = (upper).astype(int)
        upper = np.where(upper < 0, upper+360, upper)
        
        if lower > upper:
            densities[int(lower):] = camera_total
            densities[:int(upper)] = camera_total
        else:
            densities[int(lower):int(upper)+1] = camera_total
#         print(lower, upper)
        
#     break

    axs.set_thetalim(0, 2*np.pi)
#     axs.scatter(angles, totals, color=colors)
    size = 5
    axs.bar((np.arange(360)*(np.pi/180))[::size], densities[::size]/np.max(densities[::size]), 
            width=size*np.pi/180, edgecolor = "blue", color='blue')


    if axis_labels_day_ind == day_ind:
        angles = [90, 45, 0, 315, 270, 225, 180, 135]
        angles = np.array(angles)
        angles_compass = [str(a) + u"\u00b0" for a in angles]


    _ = axs.set_xticklabels(angles_compass, fontweight='bold')
    axs.set_title(f"{date}")
    _ = axs.set_rticks([.25, .5, .75, 1])
    axs.set_yticklabels([])
    
#     axs.set_rlim(0, max_bats+500)

#     axs.text(np.radians(50), 
#              3*axs.get_rmax()/4.,
#              'Bats per degree',
#             rotation=50, 
#             ha='center', va='center')
#     _ = axs.set_rticks([])
    
    

    title = f"{date} fly out fractions"

    if save:
        bf.save_fig(plots_save_folder, title, fig=fig)
    
    
# print(f"min camera {min_camera}, {min_date}: {min_bats_per_degree}")
# print(f"max camera {max_camera}, {max_date}: {max_bats_per_degree}")

In [ ]:
fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True), figsize=(20,20))


In [ ]:
np.arange(360)/()

In [ ]:
plt.scatter(np.arange(360), densities, s=1)

In [ ]:
axis_labels_day_ind = 0
save=False




center = 'middle'

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 1, subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats_per_degree = 0
max_bats_per_camera = 0
max_height = 0

zipped = zip(all_observations.items(), all_figs, all_axs)

for day_ind, ((date, day), fig, axs) in enumerate(zipped):


    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    fractions = []
    cameras = []
    totals = []
    contribution = []
    angles = []
    colors = []
    polarizations = []
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    
    print(date, len(camera_utms))

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    for camera, obs in day.items():

        fractions.append(obs['fraction_total'])
        cameras.append(camera)
        totals.append(obs['total']/360)
        angles.append(camera_angles[camera])
        colors.append(camera_colors[camera])
        polarizations.append(bf.calculate_polarization(all_tracks[date][camera]))

    max_bats_per_degree_day = np.max(totals)
    if max_bats_per_degree_day > max_bats_per_degree:
        max_bats_per_degree = max_bats_per_degree_day


    total_ind = 0
    axs.set_thetalim(-np.pi, np.pi)
    axs.scatter(angles, polarizations, color=colors)
#     if axis_labels_day_ind == day_ind:
    axs.set_xticks(angles)
    _ = axs.set_xticklabels(cameras, fontweight='bold')
    axs.set_title(f"{date}")
    
    axs.set_rlim(0, 1.0)
    
    axs.text(np.radians(0), 
             3*axs.get_rmax()/4.,
             'Polarization',
            rotation=0, 
            ha='center', va='center')
    
    title = f'{date} bat polarization center {center}'


    if save:
        bf.save_fig(plots_save_folder, title, fig=fig) 

In [ ]:


polarizations = []
days = []
for date_ind, (date, day_tracks) in enumerate(all_tracks.items()):
    for camera, tracks in day_tracks.items():
        polarizations.append(bf.calculate_polarization(tracks))
        days.append(camera)
plt.scatter(days, polarizations)

In [ ]:
date = '19Nov'

num_tracks = len(all_tracks[date]['BBC'])



draw_tracks = True

print(f"{num_tracks} tracks")

for camera, camera_tracks in all_tracks[date].items(): 
    
    if camera != "MusolePath":
        continue
    num_tracks = len(camera_tracks)
    
    p_vals = []
    for ind in range(num_tracks // 1000):
        if draw_tracks:
            plt.figure()
        tracks = []
        for track_ind in range(ind*1000, (ind+1)*1000, 50):
            track = camera_tracks[track_ind]['track']
            tracks.append(camera_tracks[track_ind])
            if draw_tracks:
                plt.scatter( track[:, 0], track[:,1], s=1)
                plt.plot([track[0, 0], track[-1, 0]], [track[0,1], track[-1, 1]])
                
                plt.scatter([track[0, 0], track[-1, 0]], 
                            [track[0,1], track[-1, 1]], c=['red', 'blue'])
                plt.gca().set_aspect('equal')
        p_vals.append(bf.calculate_polarization(tracks))
        if draw_tracks:
            plt.title(p_vals[-1])
    plt.figure()
    plt.plot(p_vals)
    plt.title(camera)

In [ ]:
len(all_tracks['19Nov']['Chyniangale'])

In [ ]:
for ind in range(9):
    tracks = all_tracks['19Nov']['Chyniangale'][ind*1000:(ind+1)*1000]
    print(bf.calculate_polarization(tracks))
    

In [ ]:
for camera, tracks in day_tracks.items():
    camera_straightness = []

    for track in tracks:
        camera_straightness.append(
            straightness(track['track']))
    plt.figure()
    plt.scatter(np.arange(len(camera_straightness)), camera_straightness, alpha=.1)

In [ ]:
np.stack([x, y], 1).shape

In [ ]:
track_ind = 1000
track = tracks[track_ind]

track = np.copy(track['track'])

plt.figure(figsize=(20,20))

plt.scatter(track[:, 0], track[:, 1], s=1)

x = np.convolve(track[:, 0], kernel, mode='valid')
y = np.convolve(track[:, 1], kernel, mode='valid')

plt.scatter(x, y, s=1)

In [ ]:
diff / np.linalg.norm(diff)

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
root_frame_folder = ".../Elements/bats"


frame_files = sorted(
    glob.glob(os.path.join(root_frame_folder,
                           date,
                           camera, 
                           "*/*.jpg"
                    )
    )
)

In [ ]:
frame_ind = 10000

frame = plt.imread(frame_files[frame_ind])

print(frame.shape)


bf.draw_tracks_on_frame(frame, frame_ind, tracks, 
                         positions=None, figure_scale=60, 
                         track_width=2, position_alpha=.5, 
                         draw_whole_track=False, shift=0)

plt.figure(figsize=(20, 20))
plt.imshow(frame)

In [ ]:
track_ind = 10002
tracks[track_ind]['track'][-1] - tracks[track_ind]['track'][0]

In [ ]:
tracks[track_ind]['track'][0]

In [ ]:
tracks[track_ind]['track'][-1]